# Notebook for make profiling of di-f Correlation experiments

## Experiment name: mxretailsalary1

## Experiment General Data
### Team roles:
* **PipeMaster**: jag.pascoe
* **BizEngineer**: 
* **DataEngineer**:
* **MLEngineer**:
* **SWEngineer**:

### Description (Use case):
Predict salary per day estimation to be obtained for working in retail sector in any state of Mexico.
Supposing you are looking for being hired in a Retail Business in any of Mexico's state you want to. You want to predict which would be the base salary per day you might get as attendant of that retail business. This salary not include any commision, tax, or any other concept.

### Type of experiment: Correlation
### Independent Variables (inputs):
1) State of Mexico where you are supposing to get hired (CAT). 
2) How many employees (including yourself) work in that particular business now (NUMBER)
3) How much sales per day in average, you estimate you will provide to that business in pesos (FLOAT)

### Dependent Variables (outputs):
1) Estimated base salary per day (FLOAT)

## Experiment preparation, imports and config.yaml

In [1]:
%load_ext autoreload
%autoreload 2
# The %load_ext autoreload and %autoreload 2 magic commands are used to automatically 
# reload modules when they are changed. This can be useful when you are developing code 
# in an interactive environment, as it allows you to see the changes you make to your modules 
# without having to restart the kernel.
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
import pandas as pd
import numpy as np
import os


# for global initialization: NOT RECOMMENDED
#initialize(version_base=None, config_path="../src/conf")
#compose(config_name='config')

with initialize(version_base=None, config_path="../src/conf"):
    cfg = compose(config_name='config')
    print(cfg)

{'general_ml': {'seed': 123, 'encoding': 'iso-8859-1', 'cloud': 'AWS'}, 'paths': {'project_dir': '/home/jagpascoe/democlient-sklearn/dif-s-mxretailsalary1', 'raw_data': '${paths.project_dir}/data/raw', 'interim_data': '${paths.project_dir}/data/interim', 'processed_data': '${paths.project_dir}/data/processed', 'reports': '${paths.project_dir}/reports', 'models': '${paths.project_dir}/models', 'api': '${paths.project_dir}/API'}, 'cloud_paths': {'bucket_path': 'dif-b-democlient-sklearn', 'experiment_path': '${cloud_paths.bucket_path}/mxretailsalary1', 'mlflow_path': '${cloud_paths.experiment_path}/mlflow', 'reports_path': '${cloud_paths.experiment_path}/reports', 'rawdata_path': '${cloud_paths.experiment_path}/raw-data', 'dvc_path': '${cloud_paths.experiment_path}/dvc-store'}, 'file_names': {'raw_file': 'raw-data.csv', 'processed_data': 'processed_data.csv', 'processed_unseen_data': 'processed_unseen_data.csv', 'data_file': 'datafile.csv', 'train_features': 'train_features.csv', 'train_l

In [2]:
#Choose the Ml model to be applied, among: regression, Classifications, time_series, Clustering, NLP
from pycaret.regression import *
from pycaret import version_
version_

'3.0.3'

## Loading the model and unseen_data

### first the model
This function loads a previously saved pipeline.

In [3]:
selected_model = load_model(os.path.join(cfg.paths.models, cfg.file_names.ml_profiling_best))
selected_model

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['income_employee_day',
                                             'employees_business'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['state'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('onehot_encoding',
                 TransformerWrapper(include=['state'],
                                    transformer=OneHotEncoder(cols=['state'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('transformation',
                 TransformerWrapper(transformer=PowerTransformer(standardize=False))),
                ('normalize', TransformerWrapper(transformer=MinMaxScaler())),
                ('actual_estimator',
                 <catboost.core.CatBoostRegressor object at 0x7f26ff600a90>)])

### then, unseen_data

In [4]:
unseen_data = pd.read_csv(os.path.join(cfg.paths.processed_data, cfg.file_names.processed_unseen_data), 
                   #encoding=cfg.general_ml.encoding,
                   )
unseen_data.head()

,state,income_employee_day,employees_business,salary_employee_day
0,Veracruz,7994.591132,5,329.596014
1,Puebla,1548.324052,4,37.371991
2,Veracruz,18250.646834,7,942.254688
3,Oaxaca,7060.781633,5,131.860879
4,Coahuila,21440.227469,10,549.892948


### now predict on unseen_data
This function generates the label using a trained model.  When data is None, it predicts label and score on the holdout set. 

In [5]:
predict_model(selected_model, data=unseen_data) 

,state,income_employee_day,employees_business,salary_employee_day,prediction_label
0,Veracruz,7994.591309,5,329.596008,251.268162
1,Puebla,1548.324097,4,37.371990,31.173436
2,Veracruz,18250.646484,7,942.254700,744.870239
3,Oaxaca,7060.781738,5,131.860886,208.920298
4,Coahuila,21440.226562,10,549.892944,552.082358
...,...,...,...,...,...
745,Oaxaca,13227.475586,6,482.449585,424.787561
746,Veracruz,10511.706055,5,450.042999,378.448351
747,Tlaxcala,10162.719727,5,361.778778,370.717574
748,Guanajuato,7328.578613,7,250.115936,305.481410


## Deployment

### as API with FastAPI
This function takes an input model and creates a POST API for inference. It only creates the API and doesn't run it automatically. 
Once you initialize API with the !python command. You can see the server on localhost:8000/docs.

In [6]:
create_api(selected_model, os.path.join(cfg.paths.api, cfg.file_names.api_ml_profiling))

type: _CURRENT_EXPERIMENT global variable is not set. Please run setup() first.

## creating docker config files
This function creates a Dockerfile and requirements.txt for productionalizing API end-point.

In [ ]:

create_docker(os.path.join(cfg.paths.api, cfg.file_names.api_ml_profiling))

Writing requirements.txt
Writing Dockerfile
Dockerfile and requirements.txt successfully created.
    To build image you have to run --> !docker image build -f "Dockerfile" -t IMAGE_NAME:IMAGE_TAG .
            


## create an app
This function creates a basic gradio app for inference. It will later be expanded for other app types such Streamlit.

In [ ]:
create_app(selected_model)

type: _CURRENT_EXPERIMENT global variable is not set. Please run setup() first.